In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

In [4]:
from sql_functions import get_dataframe

In [7]:
future_value = 142800
#num_years = 30
performance_query = '''SELECT p_ten_yr, ticker FROM public.etfs_lowrisk 
                        WHERE p_ten_yr IS NOT NULL
                        --ORDER BY p_ten_yr ASC  
                        --LIMIT 1;'''

Amount_to_invest_data = get_dataframe(performance_query)

#annual to monthly performance
Amount_to_invest_data['Monthly_Performance'] = (1 + (Amount_to_invest_data['p_ten_yr'])/100) ** (1/12) - 1

# number of months (30 years * 12 months)
#num_months = num_years * 12

# future value after 30 years

for num in range(7,38,5):
    # number of months (30 years * 12 months)
    num_months = num * 12
    monthly_amounts=[]
    for index, row in Amount_to_invest_data.iterrows():
        monthly_amount = future_value * row['Monthly_Performance'] / ((1 + row['Monthly_Performance']) ** num_months - 1)
        monthly_amounts.append(monthly_amount)
    col_name = 'MIA_' + str(num) + '_' + str(future_value)
    Amount_to_invest_data[col_name] = monthly_amounts
    Amount_to_invest_data[col_name] = Amount_to_invest_data[col_name].round(2)
print(Amount_to_invest_data)
    #print(f"Predicted future value for entry {index}: {future_value:.2f} Euros")
#etf_data['ROI_16'] = future_values
#etf_data['ROI_16'] = etf_data['ROI_16'].round(2)

     p_ten_yr ticker  Monthly_Performance  MIA_7_142800  MIA_12_142800  \
0       12.48    SPY             0.009849       1100.58         453.51   
1       12.54    IVV             0.009893       1098.27         451.74   
2       12.55    VOO             0.009901       1097.89         451.44   
3       11.98    VTI             0.009474       1119.99         468.50   
4       18.66    QQQ             0.014360        886.81         301.91   
..        ...    ...                  ...           ...            ...   
706      3.32   CHII             0.002725       1515.13         811.10   
707      0.97   EEMO             0.000805       1643.87         935.71   
708      6.00   KFYP             0.004868       1380.15         686.71   
709      4.66   CHIM             0.003803       1446.12         746.64   
710    -12.10   ERUS            -0.010690       2567.48        1939.06   

     MIA_17_142800  MIA_22_142800  MIA_27_142800  MIA_32_142800  MIA_37_142800  
0           220.30         114

In [6]:
future_value = 142800
#num_years = 30
performance_query = '''SELECT p_ten_yr, ticker FROM public.etfs_lowrisk 
                        WHERE p_ten_yr IS NOT NULL
                        --ORDER BY p_ten_yr ASC  
                        --LIMIT 1;'''

Amount_to_invest_data = get_dataframe(performance_query)

#annual to monthly performance
Amount_to_invest_data['Yearly_Performance'] =  Amount_to_invest_data['p_ten_yr']/100

# number of months (30 years * 12 months)
#num_months = num_years * 12

# future value after 30 years

for num in range(7,38,5):
    # number of months (30 years * 12 months)
    num_months = num * 12
    monthly_amounts=[]
    for index, row in Amount_to_invest_data.iterrows():
        monthly_amount = future_value * row['Yearly_Performance'] / ((1 + row['Yearly_Performance']) ** num- 1)
        monthly_amounts.append(monthly_amount)
    col_name = 'MIA_' + str(num) + '_' + str(future_value)
    Amount_to_invest_data[col_name] = monthly_amounts
    Amount_to_invest_data[col_name] = Amount_to_invest_data[col_name].round(2)
print(Amount_to_invest_data)
    #print(f"Predicted future value for entry {index}: {future_value:.2f} Euros")
#etf_data['ROI_16'] = future_values
#etf_data['ROI_16'] = etf_data['ROI_16'].round(2)

     p_ten_yr ticker  Yearly_Performance  MIA_7_142800  MIA_12_142800  \
0       12.48    SPY              0.1248      13946.31        5746.75   
1       12.54    IVV              0.1254      13920.55        5725.77   
2       12.55    VOO              0.1255      13916.27        5722.28   
3       11.98    VTI              0.1198      14162.73        5924.38   
4       18.66    QQQ              0.1866      11523.77        3923.18   
..        ...    ...                 ...           ...            ...   
706      3.32   CHII              0.0332      18456.56        9880.43   
707      0.97   EEMO              0.0097      19814.00       11278.41   
708      6.00   KFYP              0.0600      17012.48        8464.76   
709      4.66   CHIM              0.0466      17720.98        9149.52   
710    -12.10   ERUS             -0.1210      29061.29       21948.26   

     MIA_17_142800  MIA_22_142800  MIA_27_142800  MIA_32_142800  MIA_37_142800  
0          2791.66        1449.60         

In [10]:
from sql_functions import get_engine
engine = get_engine()

In [11]:
# Set the schema to your course name and the table_name variable to 'carriers_' + your initials/group number

schema = 'public' # your course schema name, example 'hh_analytics_22_1
table_name = 'etf_lowrisk_ROI_monthly_investment' # Example: 'carriers_pw' for Philipp Wendt

In [12]:
# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        Amount_to_invest_data.to_sql(name=table_name, # Name of SQL table variable
                        con= engine, # Engine or connection
                        schema= schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')

The etf_lowrisk_ROI_monthly_investment table was imported successfully.


In [1]:
def calculate_future_value(initial_investment, monthly_contribution, annual_return, expense_ratio, investment_duration_years):
    monthly_return = (1 + annual_return) ** (1/12) - 1
    total_months = investment_duration_years * 12
    total_investment = initial_investment + (monthly_contribution * total_months)
    
    for month in range(total_months):
        total_investment *= (1 + monthly_return)
        total_investment *= (1 - expense_ratio)
    
    return total_investment

# Example usage
initial_investment = 0
monthly_contribution = 1534
annual_return = 0.1248  # 8% annual return
expense_ratio = 0.09 # 0.5% annual expense ratio
investment_duration_years = 7

future_value = calculate_future_value(initial_investment, monthly_contribution, annual_return, expense_ratio, investment_duration_years)
print(f"Future value after {investment_duration_years} years: ${future_value:.2f}")


Future value after 7 years: $106.44
